In [ ]:
import os
import csv
import nltk
import re
import operator
import math
import readability
import random
from nltk import word_tokenize
from gensim.models import Word2Vec
from numpy import array
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from numpy import array
from numpy import mean
from numpy import zeros
from nltk.tag import StanfordPOSTagger
from nltk.tag import StanfordNERTagger


nltk.internals.config_java(options='-Xmx3024m')
st = StanfordPOSTagger('english-left3words-distsim.tagger', path_to_jar='stanford-postagger.jar')
ner = StanfordNERTagger('english.all.3class.distsim.crf.ser.gz', 'stanford-ner.jar')

def train_test_split(directory):
    file_names = os.listdir(directory)
    random.shuffle(file_names)
    train_list = []
    test_list = []
    i = 0
    for file_name in file_names:
        if file_name != '.DS_Store':
            if i < 4:
                train_list.append(file_name)
                i+= 1
            else:
                test_list.append(file_name)
                i = 0
    return (train_list, test_list)

def make_training_dicts(filepath, file_list):
    age_dir = {}
    time_dir = {}
    with open(filepath, encoding='latin-1') as file:
        reader = csv.reader(file)
        for row in reader:
            if row[0] in file_list:
                age_dir[row[0]] = row[1]
                time_dir[row[0]] = row[2]
    return (age_dir, time_dir)

def clean_and_tokenize_file(file_name):
    file = open(file_name)
    raw_text = file.read()
    raw_text_lower = raw_text.lower()
    caseless_tokens = nltk.word_tokenize(raw_text_lower)
    caseless_tokens = [x for x in caseless_tokens if re.match('\w+', x)]
    sentences = nltk.sent_tokenize(raw_text_lower)
    raw_sentence_tokens = nltk.sent_tokenize(raw_text)
    case_tokens = nltk.word_tokenize(raw_text)
    sentence_list = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        sentence_list.append([x for x in words if re.match('\w+', x)])
    return (caseless_tokens, sentence_list, raw_sentence_tokens, case_tokens)


def clean_and_tokenize_corpus(directory, file_list):
    file_dir = {}
    for file_name in os.listdir(directory):
        if file_name in file_list:
            file_dir[file_name] = clean_and_tokenize_file(directory + file_name)
    return file_dir


def count_total_vocab(file_wordfreqs):
    vocab = {}
    for words in file_wordfreqs:
        for word in words:
            if word in vocab:
                vocab[word] += 1
            else:
                vocab[word] = 1
    return vocab

def get_corpus_vocab(corpus, unknown_threshold):
    initial_dict = count_total_vocab(corpus)
    sorted_dict = sorted(initial_dict.items(), key=operator.itemgetter(1), reverse=True)
    filtered_dict = [key for key, value in sorted_dict[:unknown_threshold]]
    return filtered_dict

def get_idf_dict(vocab, corpus):
    idf_dict = {}
    num_docs = len(corpus) 
    for word in vocab:
        df_count = 1
        for doc in corpus:
            if word in doc:
                df_count += 1
        idf_dict[word] = math.log(num_docs / df_count)
    return idf_dict

def get_named_entities(novel):
    named_entities = ner.tag(novel)
    entities = []
    cur_entity = []
    last_tag = 'O'
    for entity in named_entities:
        if entity[1] == 'O':
            if len(cur_entity) > 0:
                new_entry = (' '.join(cur_entity), last_tag)
                if new_entry not in entities:
                    entities.append(new_entry)
                cur_entity = []
            last_tag = 'O'
        elif entity[1] == last_tag:
            cur_entity.append(entity[0])
        else:
            if len(cur_entity) > 0:
                new_entry = (' '.join(cur_entity), last_tag)
                if new_entry not in entities:
                    entities.append(new_entry)
                cur_entity = []
            cur_entity.append(entity[0])
            last_tag = entity[1]
    return entities

def get_ne_corpus(corpus):
    ne_dict = {}
    for key, value in corpus.items():
        ne_dict[key] = get_named_entities(value[3])
    return ne_dict

full_list = train_test_split('backup_plan')
train_list = full_list[0]
test_list = full_list[1]
training_data = make_training_dicts('backup.csv', train_list)
training_age = training_data[0]
training_time = training_data[1]
corpus = clean_and_tokenize_corpus('backup_plan/', train_list)
ugram_corpus = [value[0] for key, value in corpus.items()]
vocab = get_corpus_vocab(ugram_corpus, 500)
idf_dict = get_idf_dict(vocab, ugram_corpus)
ne_corpus = get_ne_corpus(corpus)
ne_vals = [value for key, value in ne_corpus.items()]
ne_vocab = get_corpus_vocab(ne_vals, 100)
ne_idf_dict = get_idf_dict(ne_vocab, ne_vals)


testing_data = make_training_dicts('backup.csv', test_list)
testing_age = testing_data[0]
testing_time = testing_data[1]
test_corpus = clean_and_tokenize_corpus('backup_plan/', test_list)
test_ugram_corpus = [value[0] for key, value in test_corpus.items()]
test_idf_dict = get_idf_dict(vocab, test_ugram_corpus)
test_ne_corpus = get_ne_corpus(test_corpus)
test_ne_vals = [value for key, value in test_ne_corpus.items()]
test_ne_idf_dict = get_idf_dict(ne_vocab, test_ne_vals)

In [16]:
def train_word2vec(corpus):
    model = Word2Vec(corpus, size=100)
    model.train(corpus, total_examples=len(corpus), epochs=50)
    return model

word2vec = train_word2vec(ugram_corpus)


NameError: name 'ugram_corpus' is not defined

In [158]:
def clean_and_tokenize_file(file_name):
    file = open(file_name)
    raw_text = file.read()
    raw_text_lower = raw_text.lower()
    caseless_tokens = nltk.word_tokenize(raw_text_lower)
    sentences = nltk.sent_tokenize(raw_text_lower)
    raw_sentence_tokens = nltk.sent_tokenize(raw_text)
    case_tokens = nltk.word_tokenize(raw_text)
    sentence_list = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        sentence_list.append([x for x in words if re.match('\w+', x)])
    return (caseless_tokens, sentence_list, raw_sentence_tokens, case_tokens)

def tf_idf(vocab, document, idf_dict):
    counts = Counter(document)
    doc_len = len(document)
    tf_idfs = []
    for word in vocab:
        doc_counter = 0
        tf = counts[word] / len(document)
        idf = idf_dict[word]
        tf_idfs.append(tf*idf)
    return tf_idfs

def get_avg_word2vec(novel):
    vec_list = []
    for word in novel:
        if word in word2vec:
            vec_list.append(word2vec[word])
    vector_array = array(vec_list)
    vector_array = numpy.mean(vector_array, axis=0)
    return vector_array.tolist()

def get_pairwise_similarity(sentences):
    sentence_vectors = []
    for sentence in sentences:
        vec_list = []
        for word in sentence:
            if word in word2vec:
                vec_list.append(word2vec[word])
        if len(vec_list) > 0:
            vector_array = array(vec_list)
            vector_array = numpy.mean(vector_array, axis=0)
            sentence_vectors.append(vector_array)
    cs_sim = cosine_similarity(sentence_vectors, sentence_vectors)
    return numpy.mean(cs_sim)

def get_pos_vector(novel, pos_tags):
    tagged_tokens = st.tag(novel)
    tags = [tag for (word, tag) in tagged_tokens]
    num_tokens = len(novel)
    pos_vector = [tags.count(pos_tag) / num_tokens for pos_tag in pos_tags]
    return pos_vector

def get_readability_score(sentences):
    newline_regex = re.compile(r'\n')
    clean_sents = []
    for sentence in sentences:
        clean_sents.append(re.sub(newline_regex, ' ', sentence))
    readability_text = '\n'.join(clean_sents)
    readability_results = readability.getmeasures(readability_text, lang='en')
    flesch_score = readability_results['readability grades']['FleschReadingEase']   
    return flesch_score

def feature_vector(novel, vocab, idf_dict, pos_tags):
    tokens = novel[0] # for unigrams
    token_sents = novel[1] #  for sent similarity
    case_tokens = novel[3] # for pos tagging and ner
    
    unigram_vector = tf_idf(vocab, tokens, idf_dict)
    pos_vector = get_pos_vector(case_tokens, pos_tags)
    word_2_vec_vector = get_avg_word2vec(tokens)
    word_vector = unigram_vector + pos_vector + word_2_vec_vector
    
    # gets ratio of unique words to total # of words
    novel_vocab = count_total_vocab([tokens])
    vocab_size = len(novel_vocab) / len(tokens)
    word_vector.append(vocab_size)
      
    cs_similarity = get_pairwise_similarity(token_sents)
    word_vector.append(cs_similarity)
    return word_vector

def feature_vector_time_period(novel, vocab, idf_dict, pos_tags, ne_vocab, ne_novel, ne_idf_dict):
    word_vector = feature_vector(novel, vocab, idf_dict, pos_tags)
    raw_sents = novel[2]
    word_vector.append(get_readability_score(raw_sents))
    ne_vector = tf_idf(ne_vocab, ne_novel, ne_idf_dict)
    time_period_vector = word_vector + ne_vector
    return time_period_vector

def create_vector_arrays(training_data, corpus, vocab, idf_dict, pos_tags, is_time, ner_vocab, ner_corpus, ner_idf_dict):
    if is_time:
        len_feature_vector = len(vocab) + len(pos_tags) + len(ner_vocab) + 103
    else:
        len_feature_vector = len(vocab) + len(pos_tags) + 102
    vector_array = zeros((len(training_data), len_feature_vector))
    results_array = zeros(len(training_data))
    index = 0
    for data, age in training_data.items():
        results_array[index] = age
        novel = corpus[data]
        if is_time:
            vector_array[index] = feature_vector_time_period(novel, vocab, idf_dict, pos_tags, ner_vocab, ner_corpus[data], ner_idf_dict)
        else:
            vector_array[index] = feature_vector(novel, vocab, idf_dict, pos_tags)
        index += 1
    return (vector_array, results_array)
    
    
tagset = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB']

training_data_time = create_vector_arrays(training_time, corpus, vocab, idf_dict, tagset, True, ne_vocab, ne_corpus, ne_idf_dict)
training_data_age = create_vector_arrays(training_age, corpus, vocab, idf_dict, pos_tags, False, None, None, None)
testing_data_age = create_vector_arrays(training_time, corpus, vocab, idf_dict, tagset, True, ne_vocab, ne_corpus, ne_idf_dict)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(array([[ 3.03641341e-03,  2.20256978e-03,  2.13857490e-03,
          2.24937200e-03,  1.46901662e-03,  1.22354375e-03,
          1.20539595e-03,  1.06307899e-03,  1.03346943e-03,
          1.48716442e-03,  7.66028170e-04,  5.19600155e-04,
          8.56767168e-04,  5.01452356e-04,  1.04779664e-03,
          9.90487796e-04,  6.71468583e-04,  6.53320784e-04,
          6.32307542e-04,  7.59342139e-04,  6.90571530e-04,
          4.89035440e-04,  5.62581786e-04,  4.68977346e-04,
          5.08138387e-04,  3.30480981e-04,  3.90655264e-04,
          2.64575814e-04,  4.51784694e-04,  4.53694989e-04,
          3.27615539e-04,  1.93894911e-04,  1.71926522e-04,
          3.07951685e-04,  2.97050824e-04,  3.17108918e-04,
          3.29525834e-04,  3.61045696e-04,  2.76037583e-04,
          2.78903025e-04,  3.64866286e-04,  2.79858172e-04,
          2.95140529e-04,  3.47673633e-04,  2.54069194e-04,
          1.93894911e-04,  2.38786836e-04,  1.95805206e-04,
          1.70016227e-04,  2.34011099e-0

In [ ]:
input_array = array(features_vector_array[0], dtype='float32')
input_vector = from_numpy(input_array)

output_array = array(features_vector_array[1], dtype='float32')
target_vector = from_numpy(output_array)

input_and_target = TensorDataset(input_vector, target_vector)

predict_array = array(test_vector, dtype='float32')
predict_vector = from_numpy(predict_array)

dickens_predict_array = array(dickens_test_array, dtype='float32')
dickens_predict_vector = from_numpy(dickens_predict_array)

linear_model = nn.Linear(115, 1)
loss_func = nn.functional.mse_loss 
optimize = optim.SGD(linear_model.parameters(), lr=0.001)
loss = loss_func(linear_model(input_vector), target_vector)

def train(num_epochs, model, loss_func, optimizer):
    for epoch in range(num_epochs):
        for xb,yb in input_and_target:
            
            # Predictions
            pred = model(xb)
            loss = loss_func(pred, yb)
            
            # Stochastic radient descent
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
    print(loss)

train(500, linear_model, loss_func, optimize)

pred = linear_model(predict_vector)
print(pred)

def goodness(predicted, actual, range_size):
    return float((predicted - actual)) / float(range_size)

print("Jane Austen")
print("Age predicted for Mansfield Park: %s" % pred)
print("Goodness Metric: %s\n" % abs(goodness(pred, 39, 15)))

pred = linear_model(dickens_predict_vector)
print("Charles Dickens")
print("Age predicted for Oliver Twist: %s" % pred)
print("Goodness Metric: %s" % abs(goodness(pred, 25, 10)))